# Нейронные сети
__Суммарное количество баллов: 10__

В этом домашнем задании можно использовать готовые классы библиотек PyTorch, Keras, TensorFlow.

### Задание 1 (4 балла)

Создайте сеть из трех сверточных слоев по 8 сверток 3х3 и двух полносвязных слоев по 64 нейрона. 

Обучите сеть на датасете mnist с тремя разными функциями активации в слоях (sigmoid, tanh, ReLU).

In [72]:
import torch
import random
import numpy as np
from PIL import Image
from keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
df = pd.read_csv('mnist.csv').dropna()

In [0]:
X, y = df[df.columns[1:]], df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = torch.Tensor(X_train.values), torch.Tensor(X_test.values), \
                                    torch.LongTensor(y_train.values), torch.LongTensor(y_test.values) 

In [0]:
X_train, X_test = X_train.reshape((X_train.shape[0], 28, 28)), X_test.reshape((X_test.shape[0], 28, 28))
X_train = X_train.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()

In [0]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

lenet5 = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
    torch.nn.Sigmoid(),
    torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
    torch.nn.Tanh(),
    torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
    torch.nn.ReLU(),
    Flatten(),
    torch.nn.Linear(28 * 28 * 8, 64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 10),
)

In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet5.parameters(), lr=1.0e-3)

In [15]:
for epoch in range(30):
  optimizer.zero_grad()
  preds = lenet5.forward(X_train)
  loss_value = loss(preds, y_train)
  loss_value.backward()
  optimizer.step()
  if epoch % 5 == 0:
    test_preds = lenet5.forward(X_test)
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    print(accuracy)

tensor(0.0846)
tensor(0.5423)
tensor(0.7519)
tensor(0.7652)
tensor(0.8182)
tensor(0.8384)


### Задание 2 (6 баллов)

Достигните точности 87% на test датасете notMNIST.

Архитектура сети может быть любая. Можно использовать Data Augmentation.

In [40]:
!unzip 'notMNIST_small.zip'

Streaming output truncated to the last 5000 lines.
 extracting: notMNIST_small/H/QmVsbENlbnQgU3ViQ2FwIEJUIFN1Yi1DYXB0aW9uLnR0Zg==.png  
 extracting: notMNIST_small/H/QmVsbENlbnRlbm5pYWwtTmFtZUFuZE51bWJlci5vdGY=.png  
 extracting: notMNIST_small/H/QmVsbGFteUV4dHJhQm9sZC5vdGY=.png  
 extracting: notMNIST_small/H/QmVsbGhvcE5GLnR0Zg==.png  
 extracting: notMNIST_small/H/QmVsd2VFRi1Cb2xkLm90Zg==.png  
 extracting: notMNIST_small/H/QmVsd2VFRi1Cb2xkQ29uZGVuc2VkLm90Zg==.png  
 extracting: notMNIST_small/H/QmVsd2VTdGQtTGlnaHRJdGFsaWMub3Rm.png  
 extracting: notMNIST_small/H/QmVsdWNpYW4tRGVtaUJvbGQub3Rm.png  
 extracting: notMNIST_small/H/QmVtYm8gTU9OT1RZUEUgRXhwZXJ0IEJvbGQgT3NGLnR0Zg==.png  
 extracting: notMNIST_small/H/QmVtYm8tU0Mub3Rm.png  
 extracting: notMNIST_small/H/QmVtYm9JbmZhbnRNVFN0ZC1Cb2xkSXQub3Rm.png  
 extracting: notMNIST_small/H/QmVtYm9UaXRsaW5nTVRTdGQub3Rm.png  
 extracting: notMNIST_small/H/QmVuZ3VpYXQgRnJpc2t5IEFUVC50dGY=.png  
 extracting: notMNIST_small/H/QmVuZ3VpYXRCUS1NZW

In [139]:
TEST_PATH = ''
directory = 'notMNIST_small'

X_test = []
y_test = []
# for each folder (holding a different set of letters)
for directory in os.listdir('notMNIST_small'):
    # for each image
    for image in os.listdir('notMNIST_small/' + directory):
        # open image and load array data
        try:
            file_path = 'notMNIST_small' + '/' + directory + '/' + image
            img = Image.open(file_path)
            img.load()
            img_data = np.asarray(img, dtype=np.int16)
            # add image to dataset
            X_test.append(img_data)
            # add label to labels
            y_test.append(directory)
        except Exception as f:
            None # do nothing if couldn't load file
            print(directory, f, image)
            break
N = len(X_test) # number of images
img_size = len(X_test[0]) # width of image
X_test = np.asarray(X_test).reshape(N, img_size, img_size,1) # add our single channel for processing purposes
y_test_cat = to_categorical(list(map(lambda x: ord(x)-ord('A'), y_test)), 10) # convert to one-hot
y_test = np.asarray(list(map(lambda x: ord(x)-ord('A'), y_test)))

F cannot identify image file 'notMNIST_small/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png' Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png
A cannot identify image file 'notMNIST_small/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png' RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png


In [0]:
X, y = X_test, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [173]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet5.parameters(), lr=1.0e-3)
batch_size = 50

test_accuracy_history = []
test_loss_history = []

X_test = X_test
y_test = y_test

for epoch in range(5):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = lenet5.forward(X_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    test_preds = lenet5.forward(X_test)
    test_loss_history.append(loss(test_preds, y_test))
    
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
    test_accuracy_history.append(accuracy)
    
    print(accuracy)

tensor(0.9535)
tensor(0.9628)
tensor(0.9674)
tensor(0.9721)
tensor(0.9628)
